In [9]:
import h5py,os
import numpy as np


path='/p/project/pines/li47/code/projectData/NST_e-scattering/pion2pt/cA2.09.48.h5'
with h5py.File(path,'w') as fw:
    path='/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_d_cA2.09.48_pion2pt.h5'
    with h5py.File(path) as f:
        for key in f.keys():
            fw.copy(f[key],fw,name=key)
            print(fw[key])
    
    path='/p/project/pines/li47/code/projectData/NST_d-tensor/data/NST_d_cA2.09.48_tensor.h5'
    with h5py.File(path) as f:
        t=f['VEV']['pi0f']['data']['pi0'][:]
        fw.copy(f['VEV']['pi0f']['data']['pi0'],fw,name='data/vev')

<HDF5 dataset "cfgs": shape (1228,), type "|O">
<HDF5 group "/data" (3 members)>
<HDF5 dataset "moms": shape (2, 3), type "<i8">


In [31]:
cfg2new=lambda cfg:{'0':'a','1':'b','2':'c','3':'d'}[cfg[-1]]+cfg[:4]
cfg2old=lambda cfg:cfg[1:]+'_r'+{'a':'0','b':'1','c':'2','d':'3'}[cfg[0]]
symExpand=lambda dat:np.concatenate([dat,np.flip(dat,axis=1)[:,1:-1]],axis=1)

for ens in ['B64','C80','D96']:
    print(ens)
    
    path2='/p/project/pines/li47/code/projectData/NST_b-discNJN/loops/'+{'B64':'cB211.072.64_2D8','C80':'cC211.060.80','D96':'cD211.054.96'}[ens]+'/'
    cfgs2=[cfg2new(cfg) for cfg in os.listdir(path2)]
    cfgs2.sort()
    
    path=f'/p/project/pines/li47/code/projectData/NST_e-scattering/pion2pt/{ens}.h5'
    with h5py.File(path,'w') as fw:
        path=f'/p/project/pines/li47/code/temp/{ens}.txt'
        with open(path,'r') as f:
            t=f.read()
            n=54
            data={}
            cfgs=[]; times=[]
            for i in range(len(t)//n):
                tt=t[i*n:(i+1)*n-1]
                tt=tt.split(' ')
                cfg=cfg2new(tt[0]); cfgs.append(cfg)
                time=int(tt[-4]); times.append(time)
                data[(cfg,time)]=tt[-3:]
        cfgs=list(set(cfgs)); times=list(set(times))
        cfgs.sort(); times.sort()
        cfgs1=cfgs
        cfgs=list(set(cfgs1).intersection(cfgs2)); cfgs.sort()
        print(len(cfgs1),len(cfgs2),len(cfgs),len(times))
        
        fw.create_dataset('cfgs',data=cfgs)
        fw.create_dataset('moms',data=[[0,0,0]])
        
        t=np.array([[np.float64(data[(cfg,time)][0]) for time in times]  for cfg in cfgs])
        t=t[:,:,None]; t=symExpand(t)
        fw.create_dataset('data/pi0_disc',data=t*2)
        print(t[0,0,0])
        
        # t_dat=[]; t_vev=[]
        # for i,cfg in enumerate(cfgs):
        #     print(i,end='         \r')
        #     with h5py.File(path2+cfg2old(cfg)+'/j.h5') as f:
        #         dic={}
        #         for i,mom in enumerate(f['mvec']):
        #             dic[tuple(mom)]=i
        #         i_0=dic[(0,0,0)]
        #         t_pi0f=f['data/j-'][:,i_0,5]*1j
        #         t_pi0f=np.complex128(t_pi0f)
        #         t_pi0i=np.conj(t_pi0f)
        #         t=np.array([np.roll(t_pi0f,-st,axis=0)*t_pi0i[st:st+1] for st in range(len(t_pi0f))])
        #         t=np.real(np.mean(t,axis=0))
                
        #         t_dat.append(t)
        #         t_vev.append(np.real(np.mean(t_pi0f)))
        # t_dat=np.array(t_dat); t_vev=np.array(t_vev)
        # t=t_dat[:,:,None]
        # fw.create_dataset('data/pi0_disc',data=t)
        # print(t_dat[0,0])
        # print(np.mean(t_vev)**2)
        # print(t_dat[0,0]-np.mean(t_vev)**2)
        
        t=np.array([[np.float64(data[(cfg,time)][1]) for time in times]  for cfg in cfgs])
        t=t[:,:,None]; t=symExpand(t)
        fw.create_dataset('data/pi0_conn',data=t)
        
        t=np.array([[np.float64(data[(cfg,time)][2]) for time in times]  for cfg in cfgs])
        t=t[:,:,None]; t=symExpand(t)
        fw.create_dataset('data/piC',data=t)
        
        fw.create_dataset('data/vev',data=np.zeros(len(cfgs)))
        # fw.create_dataset('data/vev',data=t_vev)
    print()
            

B64
751 743 743 65
0.0491563

C80
400 400 399 81
0.02179503

D96
373 496 373 97
0.0159154



In [11]:
# temp match conn & disc

import h5py,os
import numpy as np
import random

enss=['B64','C80','D96']

path='/p/project/pines/li47/code/projectData/NST_e-scattering/pion2pt_more/'

for ens in enss:
    with h5py.File(path+'base/'+ens+'.h5') as f, h5py.File(path+ens+'.h5','w') as fw:
        # print(f.keys())
        # print(f['data'].keys())
        fw.copy(f['cfgs_disc'],fw,name='cfgs')
        fw.copy(f['data/pi0_disc'],fw,name='data/pi0_disc')
        fw.copy(f['data/vev'],fw,name='data/vev')
        fw.copy(f['moms'],fw,name='moms')
        
        cfgs_conn=f['cfgs_conn'][:]
        cfgs=f['cfgs_disc'][:]
        
        dic={}
        for i,cfg in enumerate(cfgs_conn):
            dic[cfg]=i
        
        print(len(cfgs_conn),len(cfgs),len(set(cfgs)-set(cfgs_conn)))
        
        dat_conn=[]
        dat_C=[]
        for cfg in cfgs:
            if cfg in dic:
                dat_conn.append(f['data/pi0_conn'][dic[cfg]])
                dat_C.append(f['data/piC'][dic[cfg]])
            else:
                i,j=random.randint(0,len(dic)-1),random.randint(0,len(dic)-1)
                # print(i,j)
                dat_conn.append(f['data/pi0_conn'][i])
                dat_C.append(f['data/piC'][j])
        dat_conn=np.array(dat_conn)
        dat_C=np.array(dat_C)
        fw.create_dataset('data/piC',data=dat_C)
        fw.create_dataset('data/pi0_conn',data=dat_conn)


776 749 0
401 400 0
373 500 127


In [89]:
import h5py,os
import numpy as np
import random


path='/p/project/pines/li47/code/projectData/NST_e-scattering/pion2pt_more/cA2.09.48.h5'
with h5py.File(path) as f:
    print(f.keys())
    print(f['data'].keys())
    
    t=f['data/pi0_disc'][:,:,0]
    print(t[:4,:4])
    print(t[:4,:4]-np.mean(f['data/vev'])**2)

<KeysViewHDF5 ['cfgs', 'data', 'moms']>
<KeysViewHDF5 ['pi0_conn', 'pi0_disc', 'piC', 'vev']>
[[118176.78 +0.j 118173.805+0.j 118171.69 +0.j 118168.984+0.j]
 [110617.36 +0.j 110615.125+0.j 110613.06 +0.j 110610.836+0.j]
 [108774.97 +0.j 108772.46 +0.j 108771.43 +0.j 108770.49 +0.j]
 [109545.805+0.j 109542.336+0.j 109540.72 +0.j 109539.8  +0.j]]
[[ -2016.6016+0.j  -2019.5781+0.j  -2021.6953+0.j  -2024.3984+0.j]
 [ -9576.023 +0.j  -9578.258 +0.j  -9580.32  +0.j  -9582.547 +0.j]
 [-11418.414 +0.j -11420.922 +0.j -11421.953 +0.j -11422.891 +0.j]
 [-10647.578 +0.j -10651.047 +0.j -10652.664 +0.j -10653.586 +0.j]]


In [4]:
# test

import h5py,os
import numpy as np
import random


path='/p/project/pines/li47/code/projectData/NST_e-scattering/pion2pt_more/cA2.09.48.h5'
with h5py.File(path) as f:
    print(f.keys())

enss=['B64','C80','D96']

path1='/p/project/pines/li47/code/projectData/NST_e-scattering/pion2pt/'
path2='/p/project/pines/li47/code/projectData/NST_e-scattering/pion2pt_more/'

for ens in enss[:1]:
    with h5py.File(path1+ens+'.h5') as f1, h5py.File(path2+ens+'.h5') as f2:
        print(f1.keys())
        print(f1['data'].keys())
        
        ind=0
        print(f1['cfgs'][ind],f2['cfgs'][ind])
        
        NL=f1['data']['piC'].shape[1]//2
        print(NL)
        
        # ky='piC'
        # t1=f1['data'][ky][ind,:,0]; t2=f2['data'][ky][ind,:,0]
        # t=np.max(np.abs(t1/t2-1))
        # print(t)

        # ky='pi0_conn'
        # t1=f1['data'][ky][ind,:,0]; t2=f2['data'][ky][ind,:,0]
        # t=np.max(np.abs(t1/t2-1))
        # print(t)
        
        vev=np.mean(f2['data']['vev'])
        ky='pi0_disc'
        t1=f1['data'][ky][ind,:,0]; t2=(f2['data'][ky][ind,:,0]-vev**2); t3=(f2['data'][ky][ind,:,0]/2)
        print(t3)
        
    print()

<KeysViewHDF5 ['cfgs', 'data', 'moms']>
<KeysViewHDF5 ['cfgs', 'data', 'moms']>
<KeysViewHDF5 ['pi0_conn', 'pi0_disc', 'piC', 'vev']>
b'a0500' b'a0500'
64
[2829345.47073473 2829345.46210782 2829345.45489178 2829345.4495939
 2829345.44723926 2829345.44635836 2829345.44476801 2829345.4424781
 2829345.44039304 2829345.43724249 2829345.43379382 2829345.4290622
 2829345.42600869 2829345.42536459 2829345.42779368 2829345.43009495
 2829345.43142592 2829345.43080443 2829345.43025405 2829345.432196
 2829345.4338142  2829345.43494984 2829345.43518926 2829345.43603958
 2829345.43672216 2829345.43680949 2829345.43630971 2829345.43651078
 2829345.43798915 2829345.43992819 2829345.44288819 2829345.44260814
 2829345.44039865 2829345.43936693 2829345.43744319 2829345.43815037
 2829345.43911799 2829345.43866165 2829345.43836307 2829345.4370166
 2829345.43483965 2829345.4312932  2829345.42815937 2829345.42567568
 2829345.42551798 2829345.42618778 2829345.42665634 2829345.42731917
 2829345.42681777 28293